# Predicting Major Power Outages

**Jiawei Yang, Zhixing Jiang**

## Summary of Findings


### Introduction
TODO

### Baseline Model
TODO

### Final Model
TODO

### Fairness Analysis
TODO

## Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Load and Cleaning

In [29]:
outage = pd.read_excel('outage.csv')
outage.columns = [t.lower() for t in outage.columns]
outage

,obs,year,month,u.s._state,postal.code,nerc.region,climate.region,anomaly.level,climate.category,outage.start.date,...,poppct_urban,poppct_uc,popden_urban,popden_uc,popden_rural,areapct_urban,areapct_uc,pct_land,pct_water_tot,pct_water_inland
0,1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
1,2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
2,3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
3,4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
4,5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1530,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1531,1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256
1532,1533,2009,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,2009-08-29,...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256


In [30]:
state_temp = pd.read_csv('temp_state.csv').drop(columns = ['Unnamed: 0'])
state_temp.head(3)

,u.s._state,winter_avg_temp,summer_avg_temp
0,Alabama,8.1,25.9
1,Alaska,-16.3,11.3
2,Arizona,6.4,25.6


In [33]:
outage = outage.merge(state_temp, on = 'u.s._state')
outage.head(3)

,obs,year,month,u.s._state,postal.code,nerc.region,climate.region,anomaly.level,climate.category,outage.start.date,...,areapct_uc,pct_land,pct_water_tot,pct_water_inland,winter_avg_temp_x,summer_avg_temp_x,winter_avg_temp_y,summer_avg_temp_y,winter_avg_temp,summer_avg_temp
0,1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,...,0.6,91.592666,8.407334,5.478743,-10.9,19.3,-10.9,19.3,-10.9,19.3
1,2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,...,0.6,91.592666,8.407334,5.478743,-10.9,19.3,-10.9,19.3,-10.9,19.3
2,3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,...,0.6,91.592666,8.407334,5.478743,-10.9,19.3,-10.9,19.3,-10.9,19.3


### Visualization

### Baseline Model

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier

In [6]:
def remove_na(df):
    return df.loc[~df.isna().any(axis = 1).values]

In [7]:
def shuffle_df(df):
    return df.sample(frac = 1).reset_index()

In [8]:
def prepare_df(df):
    return shuffle_df(remove_na(df))

In [10]:
outage = prepare_df(outage)

In [34]:
selected_cols = ['anomaly.level', 'u.s._state', 'cause.category', 'outage.duration', 'com.price']
sub = outage[selected_cols]
sub = prepare_df(sub)
sub

,index,anomaly.level,u.s._state,cause.category,outage.duration,com.price
0,1509,0.7,Kansas,severe weather,348.0,10.18
1,312,0.3,Indiana,system operability disruption,6.0,8.99
2,452,-0.3,Washington,intentional attack,0.0,7.46
3,1371,-0.3,Vermont,intentional attack,70.0,14.63
4,731,-0.7,North Carolina,severe weather,11280.0,6.23
...,...,...,...,...,...,...
1459,160,-0.3,Michigan,severe weather,8430.0,10.50
1460,1348,-0.5,Nebraska,severe weather,60.0,7.33
1461,210,-0.9,Texas,equipment failure,318.0,9.88
1462,1119,1.0,California,islanding,26.0,16.91


In [81]:
sub['long'] = sub['outage.duration'] >= sub['outage.duration'].quantile(0.90)

In [82]:
prepro = ColumnTransformer(
    transformers = [('ohe', OneHotEncoder(handle_unknown='ignore'), ['u.s._state', 'cause.category', 'long'])],
    remainder = 'passthrough'
)
ppl_base = Pipeline(
    steps = [
        ('prepro', prepro),
        ('dtree', DecisionTreeRegressor(max_depth = 5))
    ]
)
ppl_base.fit(sub.drop(columns = ['outage.duration']), sub['outage.duration'])
ppl_base.score(sub.drop(columns = ['outage.duration']), sub['outage.duration'])

0.8653361625857243

In [83]:
hypers = {
    'dtree__max_depth':[2,3,4,5,6,7,8,9,10,11,12,13,14,15]
}

In [92]:
X_train, X_test, y_train, y_test = train_test_split(
    sub.drop(columns = ['outage.duration']),
    sub['outage.duration'],)
searcher = GridSearchCV(ppl_base, hypers, cv = 5)
searcher.fit(X_train, y_train)
searcher.best_params_, searcher.score(X_train, y_train), searcher.score(X_test, y_test)

({'dtree__max_depth': 8}, 0.9186759703371468, 0.7084784609494356)

In [79]:
sub['outage.duration'].quantile(0.95)

11246.699999999979

In [58]:
14000*0.5

7000.0

### Final Model

In [15]:
extreme_df = pd.read_csv('out.csv').drop(columns = ['Unnamed: 0'])
extreme_df = prepare_df(extreme_df)
extreme_df = extreme_df.drop(columns = ['outage.start.time'])
extreme_df.head(3)

,index,month,u.s._state,pc.realgsp.state,com.price,anomaly.level,outage.duration,winter_avg_temp,summer_avg_temp
0,196,5.0,Texas,53104,7.73,0.6,3255.0,8.8,27.3
1,741,2.0,North Carolina,44054,8.59,0.5,1950.0,5.6,24.3
2,1045,7.0,Florida,41618,7.42,0.5,221.0,15.2,27.2


In [16]:
#Function that converts winter temperature into boolean to define whether the winter is cold
def transform_cold(temp):
    df = pd.DataFrame(temp)
    def check_cold(x):
        if x <= 0 :
            return True
        else:
            return False
    return pd.DataFrame(df['winter_avg_temp'].apply(check_cold))

In [17]:
#Function that converts summer temperature into boolean to define whether the summer is hot
def transform_hot(temp):
    df = pd.DataFrame(temp)
    def check_warm(x):
        if x >= 25:
            return True
        else:
            return False
    return pd.DataFrame(df['summer_avg_temp'].apply(check_warm))

In [18]:
cold_changed = Pipeline([
        ('temp_to_cold', FunctionTransformer(transform_cold)),
        ('one_hot_encoding', OneHotEncoder())
        ]
    )

In [19]:
hot_changed = Pipeline([
        ('temp_to_hot', FunctionTransformer(transform_hot)),
        ('one_hot_encoding', OneHotEncoder())
        ]
    )

In [20]:
prepro = ColumnTransformer(
    transformers = [('ohe', OneHotEncoder(handle_unknown='ignore'), ['u.s._state']),
                   ('cold_changed_prepro', cold_changed, ['winter_avg_temp']),
                   ('hot_changed_prepro', hot_changed, ['summer_avg_temp'])],
    remainder = 'passthrough'
)
ppl_improve = Pipeline(
    steps = [
        ('prepro', prepro),
        ('dtree2', DecisionTreeRegressor())
    ]
)
ppl_improve.fit(extreme_df.drop(columns = ['outage.duration']), extreme_df['outage.duration'])
ppl_improve.score(extreme_df.drop(columns = ['outage.duration']), extreme_df['outage.duration'])

1.0

In [21]:
hypers2 = {
    'dtree2__max_depth':[2,3,4,5,6,7,8,9,10,11,12,13,14,15]
}

In [22]:
X2_train, X2_test, y2_train, y2_test = train_test_split(extreme_df.drop(columns = ['outage.duration']),
                                                    extreme_df['outage.duration'])
searcher = GridSearchCV(ppl_improve, hypers2, cv = 5)
searcher.fit(X2_train, y2_train)
searcher.best_params_, searcher.score(X2_train, y2_train), searcher.score(X2_test, y2_test)

({'dtree2__max_depth': 2}, 0.053581496195801104, -0.07582159207255756)

In [23]:
ppl_improve.fit(X2_train, y2_train)
ppl_improve.score(X2_test, y2_test)

-2.345951553137827

In [24]:
extreme_df.sample(frac = 1).reset_index()

,level_0,index,month,u.s._state,pc.realgsp.state,com.price,anomaly.level,outage.duration,winter_avg_temp,summer_avg_temp
0,643,537,2.0,Maryland,54062,11.71,-1.0,3037.0,1.5,22.9
1,278,379,8.0,Illinois,53640,9.05,1.4,90.0,-2.1,23.0
2,104,217,7.0,Texas,48322,11.58,-0.4,12124.0,8.8,27.3
3,1402,170,2.0,Texas,48595,8.93,-1.0,868.0,8.8,27.3
4,1309,106,12.0,Michigan,40097,8.97,-0.7,7650.0,-5.7,19.0
...,...,...,...,...,...,...,...,...,...,...
1459,444,436,4.0,Washington,57796,8.30,1.1,313.0,0.6,17.7
1460,1067,1473,7.0,New Hampshire,47697,13.99,-0.3,73.0,-6.1,18.6
1461,499,1447,12.0,Massachusetts,56337,10.46,0.3,115.0,-2.6,20.0
1462,722,1262,5.0,California,56365,14.87,0.8,1.0,7.9,23.0


In [25]:
outage['outage.duration']

0      3060.0
1      1620.0
2     12124.0
3      6840.0
4       488.0
5     10380.0
6      1080.0
7      1440.0
8      4140.0
9      1560.0
10     6240.0
11       15.0
12     9486.0
13      720.0
14     2460.0
15     1680.0
16      692.0
17     6713.0
18    12060.0
19      299.0
20    21540.0
21     1099.0
22    12890.0
23     2040.0
24     1890.0
25     1528.0
26     7560.0
27     1523.0
28     4200.0
29    18804.0
30      360.0
31      480.0
32     3600.0
33    17812.0
34    13920.0
35     1867.0
36      221.0
37    24780.0
38    20416.0
39    27698.0
40    12180.0
Name: outage.duration, dtype: float64

### Fairness Analysis

In [26]:
# TODO